# A

# Ex 2.4

- Decision Variables $x_{ij}$: the amount of ingredient $i$ used to produce each product $j$

- Revenue $= \sum_{j=1}^{n}\sum_{i=1}^{m}x_{ij}e_j$

- Cost $= \sum_{i=1}^{m}\sum_{j=1}^{n}x_{ij}c_i$

- Net Profit $= Revenue - Cost$

## Module Formulation

\begin{align*}
& \max \sum_{j=1}^{n}\sum_{i=1}^{m}x_{ij}e_j - \sum_{i=1}^{m}\sum_{j=1}^{n}x_{ij}c_i;\\
\\
& \sum_{j=1}^{n}x_{ij} \leq b_i,\\
& \sum_{i=1}^{m}x_{ij} \geq d_i,\\
& 0 \leq l_{ij} \leq \frac{x_{ij}}{\sum_{i=1}^{m}x_{ij}} \leq u_{ij} \leq 1,\\
\\
& x_{ij} \geq 0, b_i \geq 0, c_i \geq 0, d_i \geq 0,\\
& \forall i = 1,\ldots,m,\quad j=1,\ldots,n.\\
\end{align*}

In [1]:
%reset -f
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Input Data

In [2]:

b = np.array([1600, 2100, 1100]) # availabilities of each ingredient
c = np.array([6.4, 4.6, 2.7]) # cost of per kilo ingredient
l = np.array([[0.2,0.1,0],[0.5,0.1,0.3],[0.1,0.7,0.3]]) # lower bounds of each ingredient in a product
u = np.array([[0.5,0.2,0.3],[0.6,0.3,0.4],[0.2,0.8,0.5]]) # upper bounds of each ingredient in a product
e = np.array([5.7, 5.2, 4.3]) # revenue of per kilo product
d = np.array([200, 1000, 500]) # minimum production demand


In [3]:
M = l.shape[0]
N = l.shape[1]
print(M, N)

3 3


In [4]:
model = gp.Model()
model.reset()
x = model.addVars(M, N, lb=0.0) # the amount of each ingredient in a product

netprofitobjecttive = model.setObjective( (sum(x[i,j]*e[j] for i in range(0,M) for j in range(0,N)))
                                         -(sum(x[i,j]*c[i] for j in range(0,N) for i in range(0,M))), GRB.MAXIMIZE) 

Academic license - for non-commercial use only - expires 2022-09-03
Using license file /Users/changli/gurobi.lic
Discarded solution information


## Constraints

In [5]:
# usage shall not exceed avalibility
for i in range(0, M):
    model.addConstr(quicksum(x[i,j] for j in range(0,N)) <= b[i], name='i'+str(i) )

# lower bounds of proportion
for j in range(0, N):
    for i in range(0, M):
        model.addConstr(x[i,j] >= quicksum(x[i,j] for i in range(0,M))*l[i,j] , name='l'+str(i)+str(j))

# upper bounds of proportion
for j in range(0, N):
    for i in range(0, M):
        model.addConstr(x[i,j] <= quicksum(x[i,j] for i in range(0,M))*u[i,j] , name='u'+str(i)+str(j))
        
# minimum production demand
for j in range(0, N):
    model.addConstr( quicksum(x[i,j] for i in range(0,M)) >=  d[j], name='d'+str(j))

# overall weight of products shall not exceed that of ingredients
model.addConstr(quicksum(x[i,j] for i in range(0,M) for j in range(0,N)) <= quicksum(b[i] for i in range(0,M)), name='p')

model.update()



## Results

In [6]:
model.optimize()
if model.status != GRB.Status.OPTIMAL:
    print("***** Gurobi solve status:", model.status)
    print("***** This is a problem. Model does not have an optimal solution")
    raise StopExecution

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 25 rows, 9 columns and 79 nonzeros
Model fingerprint: 0x81ca4bb5
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+03]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 24 rows, 9 columns, 78 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1100000e+03   7.810713e+02   0.000000e+00      0s
       8    3.4900000e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds
Optimal objective  3.490000000e+03


In [7]:
solution = model.X
print("************************Solutions************************")
s= np.array(solution).reshape(M,N)
print('{0:<1}\t\t{1:<8}\t\t{2:<8}\t\t{3:<1}\t\t{4:<8}'.format('','ingredient','product','','weight (kg)'))
for i in range(0,M):
    for j in range(0,N):
        #print("{} {} {} = {}".format('x',i+1,j+1,round(s[i,j])))
        print(("{0:<1}\t\t{1:<8}\t\t{2:<8}\t\t{3:<1}\t\t{4:<8}").format("x",i+1,j+1,'=',round(s[i,j]),2))      

************************Solutions************************
 		ingredient		product 		 		weight (kg)
x		1       		1       		=		750     
x		1       		2       		=		100     
x		1       		3       		=		150     
x		2       		1       		=		1500    
x		2       		2       		=		200     
x		2       		3       		=		200     
x		3       		1       		=		250     
x		3       		2       		=		700     
x		3       		3       		=		150     
